<a href="https://colab.research.google.com/github/Samin-Sadaf7/QuestionAnswerGenerator/blob/main/PDFreader_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall typer -y
!pip install --force-reinstall spacy weasel

In [ ]:
!pip install --force-reinstall --no-deps typer==0.9.0
!pip install fastapi-cli
!pip install langchain-community sentence-transformers pypdf2 pypdf chromadb
!pip install google-generativeai

In [ ]:
!pip install pandas

In [3]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
import os
import textwrap

In [4]:
def split_pdf(file_path, chunk_size, chunk_overlap):
    loader = PyPDFLoader(file_path)
    pages = loader.load_and_split()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs = text_splitter.split_documents(pages)
    return docs

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
pdf_path = '/content/gdrive/MyDrive/AI_integration_kernel/vishram Singh upper limb and thorax v1-1.pdf'

In [ ]:
docs =split_pdf(pdf_path, chunk_size=3000, chunk_overlap=50)

In [ ]:
import pandas as pd

In [ ]:
# Prepare data for DataFrame
data = []
for doc in docs:
    data.append({
        'page_content': doc.page_content,
        'source': doc.metadata['source'],
        'page': doc.metadata['page'],
        'flag': 0
    })

# Create DataFrame
df = pd.DataFrame(data)

In [6]:
!pip install --quiet pandas pymongo sentence_transformers transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 11.5 MB/s eta 0:00:00


In [7]:
!python -m pip install "pymongo[srv]"

In [8]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

from google.colab import userdata
uri = userdata.get('MongoDB_URL')

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [9]:
db = client['RAG_application']

In [10]:
{
    "fields":[{
        "numDimensions" : 1024,
        "path" : "embedding",
        "similarity": "cosine",
        "type": "vector"
    }]
}

{'fields': [{'numDimensions': 1024,
   'path': 'embedding',
   'similarity': 'cosine',
   'type': 'vector'}]}

In [11]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("thenlper/gte-large")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [12]:
def get_embedding(text:str)->list[float]:
  if not text.strip():
    print("Attemted to do embedding on an empty text")
    return []
  embedding = embedding_model.encode(text)
  return embedding.tolist()

In [89]:
def vector_search(user_query, collection):
    query_embedding = get_embedding(user_query)
    print(query_embedding)

    if query_embedding is None:
        return "Invalid Query Embedding"

    pipeline = [
        {
            "$vectorSearch": {
                "index": "vector_index",
                "queryVector": query_embedding,
                "path": "embedding",
                "numCandidates": 10,
                "limit": 3,
            }
        },
        {
            "$project": {
                "page_content": 1,
                "flag": 1,
                "_id": 1,
                "score": {"$meta": "vectorSearchScore"},
            }
        },
        {
            "$match": {
                "flag": 0,
                "$expr": {
                    "$gte": [{"$strLenCP": "$page_content"}, 100]
                }
            }
        },
    ]

    # Run the aggregation pipeline
    result = list(collection.aggregate(pipeline))

    # Extract document IDs from the result
    document_ids = [doc["_id"] for doc in result]

    # Update the flag value to 1 for the matched documents
    if document_ids:
        update_result = collection.update_many(
            {"_id": {"$in": document_ids}},
            {"$set": {"flag": 1}}
        )
        print(f"Updated {update_result.modified_count} documents")

    return result

In [90]:
def get_knowledge(user_query, collection):
  get_knowledge = vector_search(user_query, collection)
  return get_knowledge

In [91]:
def get_search_result(get_knowledge):
  search_result = ""

  for result in get_knowledge:
    score = result.get('score')
    print(score)
    text = result.get('page_content',' ')
    text = textwrap.fill(text, width=100)
    print(text)
    text_length = len(text)
    print(text_length)
    if score > 0.8 and text_length > 200:
     search_result += f"{text}\n"

  return search_result

In [ ]:
df.head()

,page_content,source,page,flag
0,TEXTBOOK OF ANATOMY UPPER LIMB AND THORAX\nPre...,/content/gdrive/MyDrive/AI_integration_kernel/...,1,0
1,Prelims.indd ii5/8/2014 4:01:45 PM,/content/gdrive/MyDrive/AI_integration_kernel/...,2,0
2,ELSEVIERA division ofReed Elsevier India Priva...,/content/gdrive/MyDrive/AI_integration_kernel/...,3,0
3,"T extbook of Anatomy: Upper Limb and Thorax, V...",/content/gdrive/MyDrive/AI_integration_kernel/...,4,0
4,Dedicated toMy MotherLate Smt Ganga Devi Singh...,/content/gdrive/MyDrive/AI_integration_kernel/...,5,0


In [ ]:
df['embedding'] = df['page_content'].apply(get_embedding)

In [ ]:
documents = df.to_dict('records')

In [ ]:
import json
documents_json = json.dumps(documents)
with open('documents.json', 'w') as f:
    f.write(documents_json)

In [ ]:
len(documents)

520

In [ ]:
result = db.Medical_Books_RAG.insert_many(documents)
print(f"Inserted {len(result.inserted_ids)} documents into the collection.")

Inserted 520 documents into the collection.


In [16]:
book_collection = db['Medical_Books_RAG']

In [ ]:
query = 'clavicle '
knowledge = get_knowledge(query, book_collection)
context = get_search_result(knowledge)

In [93]:
document_ids = [doc["_id"] for doc in knowledge]

# Update the flag value to 1 for the matched documents
if document_ids:
  update_result = book_collection.update_many(
            {"_id": {"$in": document_ids}},
            {"$set": {"flag": 0}}
   )
  print(f"Updated {update_result.modified_count} documents")

Updated 3 documents


In [99]:
collection = db['topic']

In [ ]:
import re
topic = 'Thoracic wall, intercostal space, thoracic inlet'
regex = re.compile(rf'^\s*{topic}\s*$', re.IGNORECASE)

# Query to find documents where the topic or subtopic matches the regex
query = {
    "$or": [
        {"topic": regex},
        {"subtopic": regex}
    ]
}

# Execute the query and save the results into a list
results = list(collection.find(query))
print(results)